# Spark DataFrame Missing Data

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("spark://spark:7077").appName("Missing").getOrCreate()

In [ ]:
df = spark.read.csv("/data/ContainsNull.csv",header=True, inferSchema=True)

In [ ]:
df.show()

## Dropping Null Values

In [ ]:
# Use .na.drop() to drop rows with any null values
df.na.drop().show()

In [ ]:
# Use .na.drop(thresh=<n>) to drop rows with at least <n> null values
df.na.drop(thresh=2).show()

In [ ]:
# Use .na.drop(how=<"any"|"all">) to drop rows with any null values or all null values
df.na.drop(how="all").show()


In [ ]:
# Use .na.drop(subset=<[cols]>) to drop rows with null values in subset of columns
df.na.drop(subset="Sales").show()

In [ ]:
# Filling Null Values

In [ ]:
df.printSchema()

In [ ]:
# Spark knows to automatically fill string columns with string values amd numeric columns with numeric values
df.na.fill(0).show()
# Does not fill string column

In [ ]:
# Fill specific columns with specific value
df.na.fill("No Name", subset=["Name"]).show()

In [ ]:
# Using computed mean value to fill column
from pyspark.sql.functions import mean

In [ ]:
mean_val = df.select(mean(df["Sales"])).collect()

In [ ]:
# .collect() returns list of rows
mean_val

In [ ]:
print(mean_val[0][0])
mean_sales = mean_val[0][0]

In [ ]:
df.na.fill(mean_sales, subset=["Sales"]).show()